# Περιγραφή

Δημιουργία του **πρωτόλειου DataFrame** που περιέχει όλα τα διαθέσιμα δεδομένα. Η μετατροπή της μορφοποίησης των δεδομένων γίνεται για λόγους **ευχρηστίας**. Τέλος πραγματοποιείται η εξαγωγή του DataFrame σε **csv** αρχείο, με σκοπό την εύκολη και γρήγορη χρήση του στο μέλλον.

Σε αυτό το notebook συγκεντρώθηκαν τα δεδομένα από όλα τα xml αρχεία που έχουμε στη διάθεση μας. Κάθε xml αρχείο αντιστοιχεί και σ' έναν υπο εξέταση άνθρωπο (subject). Η ανάπτυξη του κώδικα στο συγκεκριμένο notebook βασίστηκε στο *xml_to_df_example.ipynb*, το οποίο περιγράφει τη διαδικασία μετατροπής των δεδομένων, που αναφέρονται σε έναν μόνο άνθρωπο.

# Εισαγωγή βιβλιοθηκών

In [78]:
import os
import xml.etree.ElementTree as et
import pandas as pd

# Επεξεργασία δεδομένων

## Επεξεργασία xml αρχείων

Κάθε xml αρχείο αντιστοιχεί σε έναν υπό εξέταση άνθρωπο (subject)

In [79]:
def check_if_NoneType(child):
    '''
    Avoid AttributeError: 'NoneType' object has no attribute 'text'
    '''
    if writing.find(child) is not None:
        return writing.find(child).text
    else:
        return 'NaN'

In [80]:
xml_path = '../Data/data/'
df_list  = []

for file in os.listdir(xml_path):
    tree = et.parse(xml_path+file)
    root = tree.getroot()
    
    for ID in root.findall('ID'):
        identity = ID.text
        
    datetimes = []
    titles    = []
    texts     = []
    infos     = []
    
    for writing in root.findall('WRITING'):
        datetimes.append(check_if_NoneType('DATE'))
        titles.append(check_if_NoneType('TITLE'))
        texts.append(check_if_NoneType('TEXT'))
        infos.append(check_if_NoneType('INFO'))
    
    zippedList     = list(zip(datetimes, titles, texts, infos))
    df             = pd.DataFrame(data=zippedList, columns=['DateTime', 'Title', 'Text', 'Info'])
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df['Subject']  = identity
    
    df_list.append(df)

In [81]:
# Concatenate all DataFrames. Each DataFrame is referred to one subject.
all_data_df = pd.concat(df_list, axis=0)

# Create col Post_ID with values [0 ... total_num_of_rows]
num_of_rows            = all_data_df.shape[0]
all_data_df['Post_ID'] = [i for i in range(num_of_rows)]

# Set Post_ID as index of the DataFrame
all_data_df.set_index('Post_ID', inplace=True)
all_data_df.tail(8)

,DateTime,Title,Text,Info,Subject
Post_ID,,,,,
170425,2018-09-01 15:40:33,NaN,That is actually much better,Reddit post,subject6918
170426,2018-09-02 02:37:56,"My wife told me, #8220;If anything happens to...",NaN,Reddit post,subject6918
170427,2018-09-02 03:03:01,SiCk PsYcHo FuCkInG DeStRoYS BrOtHeRs EaR,NaN,Reddit post,subject6918
170428,2018-09-02 04:57:57,NaN,F,Reddit post,subject6918
170429,2018-09-02 14:08:42,"Leaving your bike in the pasture, WCGW?",NaN,Reddit post,subject6918
170430,2018-09-02 14:11:54,Wcgw if i kick the stairs,NaN,Reddit post,subject6918
170431,2018-09-02 14:26:26,Whatcouldgowrong if i bounce a ball and a bottle?,NaN,Reddit post,subject6918
170432,2018-09-03 02:47:10,Goofing off and not working.,NaN,Reddit post,subject6918


## Επεξεργασία του txt αρχείου

Το txt αρχείο περιέχει τη **γενική αλήθεια** (golden truth). Δηλαδή, σημειώνει για κάθε υπο εξέταση άνθρωπο αν εγκυμονεί ο κίνδυνος να αυτο-τραυματισεί. Αυτός θα είναι και ο **στόχος** (target) των μοντέλων που θα αναπτυχθούν. Το πλήθος των υπό εξέταση ανθρώπων είνα **340**

|                             | Σήμανση |
|:---------------------------:|:-------:|
|  Κίνδυνος αυτο-τραυματισμού |    1    |
|    Δεν υπάρχει κίνδυνος     |    0    |

In [82]:
txt_path      = '../Data/golden_truth.txt'
df_gt         = pd.read_csv(txt_path, sep=" ", header=None)
df_gt.columns = ['Subject', 'Target']

df_gt.tail(3)

,Subject,Target
337,subject3270,0
338,subject6464,0
339,subject8721,0



|                                     | Πλήθος |
|:-----------------------------------:|:------:|
|  Κινδυνεύουν να αυτο-τραυματιστούν  |  299   |
|      Δεν κινδυνεύουν                |   41   |

In [83]:
df_gt['Target'].value_counts()

0    299
1     41
Name: Target, dtype: int64

# Ένωση των DataFrames

Ένωση του df που περιέχει όλα τα δεδομένα σχετικά με τις δημοσιεύσεις των υπό εξέταση ανθρώπων με εκείνο που περιέχει τα δεδομένα για την κατάσταση τους.

In [84]:
complete_df = pd.merge(all_data_df, df_gt)

complete_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170433 entries, 0 to 170432
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   DateTime  170433 non-null  datetime64[ns]
 1   Title     170433 non-null  object        
 2   Text      170433 non-null  object        
 3   Info      170433 non-null  object        
 4   Subject   170433 non-null  object        
 5   Target    170433 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 9.1+ MB


In [85]:
complete_df.head(4)

,DateTime,Title,Text,Info,Subject,Target
0,2016-08-18 20:55:25,Quebec to recieve 10 billion of 18 billion dol...,NaN,Reddit post,subject3178,0
1,2016-08-25 02:07:48,What are some lesser known tactics to get a be...,NaN,Reddit post,subject3178,0
2,2016-09-03 03:51:40,What are some old sayings that aren't very pop...,NaN,Reddit post,subject3178,0
3,2016-09-06 00:30:20,(Canada)Just starting fourth year: is there an...,Is there any material/study questions anyone t...,Reddit post,subject3178,0


# Εξαγωγή

Εξαγωγή του DataFrame σε csv αρχείο.

In [86]:
csv_path = '../Data/CSVs/first_complete_df.csv'
complete_df.to_csv(csv_path)